In [1]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)
model.T = pe.Set(initialize = transshipment_points ,ordered = False)
model.C = pe.Set(initialize = customers ,ordered = False)
model.M = pe.Set(initialize = modes ,ordered = False)
model.P = pe.Set(initialize = periods, ordered = False)

# Decision Variables
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # direct tonnes                         !!!TODO : meer uitleg !!!
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # indirect tonnes                       !!!TODO : meer uitleg !!!
model.q_bl = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeReals)                # quantity block mode                   !!!TODO : meer uitleg !!!  
model.q_s_w = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeReals)               # quantity single wagon mode            !!!TODO : meer uitleg !!!
model.q_ss = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeReals)                # quantity shortsea ship mode           !!!TODO : meer uitleg !!!
model.q_ba = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeReals)                # quantity barge mode                   !!!TODO : meer uitleg !!!
model.td = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route     !!!TODO : meer uitleg !!!
model.tid = pe.Var(model.F, model.T, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route   !!!TODO : meer uitleg !!!

In [9]:
#block_trans single_trans shortsea_trans barge_trans truck_fixed_cost
objExpr = sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C for p in model.P)     \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans for p in model.P)            \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans for p in model.P)         \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans for p in model.P)            \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans for p in model.P)                  \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C for p in model.P)                        \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C for p in model.P)

model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

KeyError: "Index '('Ijmuiden', 'Rhenues logistics', 1)' is not valid for indexed component 'q_ba'"

In [ ]:
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] + model.q_s_w[i,j,p] + model.q_ss[i,j,p] + model.q_ba[i,j,p] for j in model.T) <= production_cap[i]
        model.prodCap.add(expression)

model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j]
        model.meetDemand.add(expression)

model.flowBalance = pe.ConstraintList()
for p in model.P:
    for j in model.T:
        expression = sum(model.q_bl[i,j,p] + model.q_s_w[i,j,p] + model.q_ss[i,j,p] + model.q_ba[i,j,p] for i in model.F) == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)

In [ ]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True)